In [2]:
# imports
import torch
import torch.nn.functional as F
import torch.nn as nn
import pandas as pd

import torch.optim as optim
import time
from opacus import PrivacyEngine
from vantage6.tools.util import info, warn
from torchvision import transforms
import argparse
from torchvision import datasets, transforms

In [3]:
# simple model 

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output


In [ ]:
# to check the params (tensors)

# model = Net()

# for parameter in model.parameters():
#     print(parameter)

In [4]:
# initialises training

def RPC_initialize_training(data, gamma, learning_rate, local_dp):
    """
    Initializes the model, optimizer and scheduler and shares the parameters
    with all the workers in the group.

    This should be sent from server to all nodes.

    Args:
        data: contains the local data from the node
        gamma: Learning rate step gamma (default: 0.7)
        learning_rate: The learning rate for training.
        cuda: Should we use CUDA?
        local_dp: bool whether to apply local_dp or not.

    Returns:
        Returns the device, model, optimizer and scheduler.
    """
    
    # Determine the device to train on
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    # print("\033[0;{};49m Rank {} is training on {}".format(device))

    # Initialize model and send parameters of server to all workers
    model = Net()
    model.to(device)

    # intializing optimizer and scheduler
    optimizer = optim.SGD(model.parameters(), lr=learning_rate)

    # adding DP if true
    if local_dp == True:
        privacy_engine = PrivacyEngine(model, batch_size=64,
                sample_size=60000, alphas=range(2,32), noise_multiplier=1.3,
                max_grad_norm=1.0,)
        privacy_engine.attach(optimizer)

    # returns device, model, optimizer which will be needed in train and test
    return device, model, optimizer    

In [10]:
# basic training of the model

# Question: train gets model, device, optimizer from initialize_training, which is specified within train function, 
# why do I need to call it again before executing the function? Because in vantage6 when I sent the tasks I cannot define that but only in the master function


def RPC_train(data, log_interval, local_dp, epoch, round, delta=1e-5):
    """
    Training the model on all batches.
    Args:
        epoch: The number of the epoch the training is in.
        round: The number of the round the training is in.
        log_interval: The amount of rounds before logging intermediate loss.
        local_dp: Training with local DP?
        delta: The delta value of DP to aim for (default: 1e-5).
    """
    # loading arguments/parameters from first RPC_method
    device, model, optimizer = RPC_initialize_training(data, gamma, learning_rate, local_dp) # is this allowed in vantage6? calling one RPC_method in another?
     
    train_loader = data
    
    model.train()
# for batch_idx, (data, target) in enumarate(train_loader):
    for batch_idx, data in enumerate(train_loader, 0): 
        data, target = data
        # Send the data and target to the device (cpu/gpu) the model is at
        data, target = data.to(device), target.to(device)
        # Clear gradient buffers
        optimizer.zero_grad()
        # Run the model on the data
        output = model(data)
        # Calculate the loss
        loss = F.nll_loss(output, target)
        # Calculate the gradients
        loss.backward()
        # Update model
        optimizer.step()
        
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                 100. * batch_idx / len(train_loader), loss.item()))

#     Adding differential privacy or not
    if local_dp == True:  
        epsilon, alpha = optimizer.privacy_engine.get_privacy_spent(delta)
            # print("\033[0;{};49m Epsilon {}, best alpha {}".format(epsilon, alpha))
     
    
    


# # This function trains the neural network for one epoch
# def train(epoch):
#     model.train()
#     for batch_idx, (data, target) in enumerate(train_loader):
#         # Move the input and target data on the GPU
#         data, target = data.to(device), target.to(device)
#         # Zero out gradients from previous step
#         optimizer.zero_grad()
#         # Forward pass of the neural net
#         output = model(data)
#         # Calculation of the loss function
#         loss = F.nll_loss(output, target)
#         # Backward pass (gradient computation)
#         loss.backward()
#         # Adjusting the parameters according to the loss function
#         optimizer.step()
#         if batch_idx % 10 == 0:
#             print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
#                 epoch, batch_idx * len(data), len(train_loader.dataset),
#                 100. * batch_idx / len(train_loader), loss.item()))



In [ ]:
# Model Evaluation

def RPC_test(data):
    """
    Tests the model.

    Args:
        color: The color for the terminal output for this worker.
        model: The model to test.
        device: The device to test the model on.
        test_loader: The local loader for test local. -> no inside function
    """


#     test_loader = torch.load("./testing.pt")
    test_loader = torch.utils.data.DataLoader(datasets.MNIST('../mnist_data', 
                                                          download=True, 
                                                              train=False,
                                                          transform=transforms.Compose([
                                                              transforms.ToTensor(), # first, convert image to PyTorch tensor
                                                              transforms.Normalize((0.1307,), (0.3081,)) # normalize inputs
                                                          ])), 
                                           batch_size=10, 
                                           shuffle=True)

    device, model, optimizer = RPC_initialize_training(data, gamma, learning_rate, local_dp)

    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            # Send the local and target to the device (cpu/gpu) the model is at
            data, target = data.to(device), target.to(device)
            # Run the model on the local
            output = model(data)
            # Calculate the loss
            test_loss += F.nll_loss(output, target, reduction='sum').item()
            # Check whether prediction was correct
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print(test_loss)

    # print('\033[0;{};49m \nAverage loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    #         test_loss, correct, len(test_loader.dataset),
    #         100. * correct / len(test_loader.dataset)))

In [ ]:
# FedAvg gathering of parameters 

def RPC_get_parameters(data, model, parameters):
    """
    Get parameters from nodes
    """
    data_size = len(data) // 3 # number of nodes# size of dataset
    
    weights = []
    # Gather the data sizes on the server
    tensor_weights = torch.tensor(data_size)
    tensor_weights = tensor_weights[1:]
    # Convert all tensors back to weights
    for tensor in tensor_weights:
            weights.append(tensor.item())

            
    for parameters in model.parameters():
        return {
            "params": parameters,
        }


"""
this might need to be combined with training, so that train 
returns the parameters or that it at least calls the results of training function
"""



In [ ]:
# averaging of returned parameters 

def average_parameters(data, model):
    """
    Get parameters from nodes and calculate the average
    :param model: torch model
    :param parameters: parameters of model
    :param weights:
    :return:
    """
    
    parameters = RPC_get_parameters() # makes returned parameters from RPC_get_parameters the parameters used in this function

    # TODO: local: since we usually just get the parameters, this well be an entire task, therefore, we might need to train for each individually


    with torch.no_grad():
        for parameters in model.parameters():
            average = sum(x * y for x, y in zip(parameters[i], weights)) / sum(weights)
            parameters.data = average
            i = i + 1
        return {
            "params_averaged": model
        }
    

#     i = 0
#     with torch.no_grad():
#     for param in model.parameters():
#     # The first entry of the provided parameters when using dist.gather
#     # method also contains the value from the server, remove that one
#     minus_server = parameters[i][1:]
#     # Calculate the average by summing and dividing by the number of
#     # workers
#     s = sum(minus_server)
#     average = s/len(minus_server)
#     # Change the parameter of the global model to the average
#     param.data = average
#     i = i + 1

In [ ]:
# training with those averaged parameters

def RPC_fed_avg(data, local_dp, model, device, optimizer, epoch, delta=1e-5):
    """
    Training and testing the model on the workers concurrently using federated
    averaging, which means calculating the average of the local model
    parameters after a number of (local) epochs each training round.

    In vantage6, this method will be the training of the model with the average parameters (weighted)

    Returns:
        Returns the final model
    """
    # TODO: local: since we usually just get the parameters, this well be an entire task, therefore, we might need to train for each individually
    model = RPC_average_parameters()
    
    for epoch in range(1, epoch + 1):
        # Train the model on the workers again
        RPC_train(data, local_dp, model, device, optimizer, epoch, delta=1e-5)
        # Test the model on the workers
        RPC_test(data, model, device)

    gather_params = model.get_parameters() # or model.parameters()

    RPC_train(model.RPC_average_parameters_weighted(gather_params))

    return model, parameters


    ## OR 

#     parameters = RPC_average_parameters_weighted(data, model, parameters, weights) # then uses those parameters for training



        # # Gather the parameters after the training round on the server
        #     gather_params = coor.gather_parameters(rank, model, group_size + 1, subgroup)
        #
        #     # If the server
        #     if rank == 0:
        #         # Calculate the average of the parameters and adjust global model
        #         coor.average_parameters_weighted(model, gather_params, weights)
        #
        #     # Send the new model parameters to the workers
        #     coor.broadcast_parameters(model, group)


In [11]:
"""
These are the parameters needed for the function
Data loading and transforming (this will be done beforehand 
and then stored in './local/training.pt' and './testing.pt')
"""

learning_rate=0.01

gamma=0.7

data = torch.utils.data.DataLoader(datasets.MNIST('../mnist_data', 
                                                          download=True,
                                                          train=True,
                                                          transform=transforms.Compose([
                                                              transforms.ToTensor(), # first, convert image to PyTorch tensor
                                                              transforms.Normalize((0.1307,), (0.3081,)) # normalize inputs
                                                          ])), 
                                           batch_size=10, 
                                           shuffle=True)

local_dp = True


# optimizer = optim.SGD(model.parameters(), lr=learning_rate)

epoch = 1

round = 1

"\nTODO, the RPC_initialize_training method is \ncalled in the RPC_train method, \nand yet it doesn't know where to get model, device, optimizer from. \n"

In [12]:
# local_dp = False
# RPC_initialize_training(data, gamma, learning_rate, local_dp)

log_interval = 5

# use_cuda = torch.cuda.is_available()
# device = torch.device("cuda" if use_cuda else "cpu")

# model = Net()
# optimizer = optim.SGD(model.parameters(), lr=learning_rate)

RPC_train(data, log_interval, local_dp, epoch, round)

# RPC_test(data)


/Users/simontokloth/anaconda3/envs/ppsdg/lib/python3.7/site-packages/opacus/privacy_engine.py:518: UserWarning: The sample rate will be defined from ``batch_size`` and ``sample_size``.The returned privacy budget will be incorrect.
  "The sample rate will be defined from ``batch_size`` and ``sample_size``."
/Users/simontokloth/anaconda3/envs/ppsdg/lib/python3.7/site-packages/opacus/privacy_engine.py:195: UserWarning: Secure RNG turned off. This is perfectly fine for experimentation as it allows for much faster training performance, but remember to turn it on and retrain one last time before production with ``secure_rng`` turned on.
  "Secure RNG turned off. This is perfectly fine for experimentation as it allows "
/Users/simontokloth/anaconda3/envs/ppsdg/lib/python3.7/site-packages/torch/nn/modules/module.py:795: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing so

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.313097
Train Epoch: 1 [50/60000 (0%)]	Loss: 2.338604
Train Epoch: 1 [100/60000 (0%)]	Loss: 2.277131
Train Epoch: 1 [150/60000 (0%)]	Loss: 2.299821
Train Epoch: 1 [200/60000 (0%)]	Loss: 2.292620
Train Epoch: 1 [250/60000 (0%)]	Loss: 2.256905
Train Epoch: 1 [300/60000 (0%)]	Loss: 2.276335
Train Epoch: 1 [350/60000 (1%)]	Loss: 2.327846
Train Epoch: 1 [400/60000 (1%)]	Loss: 2.372798
Train Epoch: 1 [450/60000 (1%)]	Loss: 2.281914
Train Epoch: 1 [500/60000 (1%)]	Loss: 2.356257
Train Epoch: 1 [550/60000 (1%)]	Loss: 2.279763
Train Epoch: 1 [600/60000 (1%)]	Loss: 2.246746
Train Epoch: 1 [650/60000 (1%)]	Loss: 2.389159
Train Epoch: 1 [700/60000 (1%)]	Loss: 2.345253
Train Epoch: 1 [750/60000 (1%)]	Loss: 2.185812
Train Epoch: 1 [800/60000 (1%)]	Loss: 2.302603
Train Epoch: 1 [850/60000 (1%)]	Loss: 2.205883
Train Epoch: 1 [900/60000 (2%)]	Loss: 2.197359
Train Epoch: 1 [950/60000 (2%)]	Loss: 2.287045
Train Epoch: 1 [1000/60000 (2%)]	Loss: 2.281614
Train Epoch: 1 

Train Epoch: 1 [8500/60000 (14%)]	Loss: 2.248651
Train Epoch: 1 [8550/60000 (14%)]	Loss: 2.456940
Train Epoch: 1 [8600/60000 (14%)]	Loss: 2.071569
Train Epoch: 1 [8650/60000 (14%)]	Loss: 1.958503
Train Epoch: 1 [8700/60000 (14%)]	Loss: 2.096431
Train Epoch: 1 [8750/60000 (15%)]	Loss: 2.321998
Train Epoch: 1 [8800/60000 (15%)]	Loss: 2.328505
Train Epoch: 1 [8850/60000 (15%)]	Loss: 2.347884
Train Epoch: 1 [8900/60000 (15%)]	Loss: 1.588742
Train Epoch: 1 [8950/60000 (15%)]	Loss: 1.949710
Train Epoch: 1 [9000/60000 (15%)]	Loss: 2.130940
Train Epoch: 1 [9050/60000 (15%)]	Loss: 2.076988
Train Epoch: 1 [9100/60000 (15%)]	Loss: 2.505519
Train Epoch: 1 [9150/60000 (15%)]	Loss: 1.711868
Train Epoch: 1 [9200/60000 (15%)]	Loss: 2.649544
Train Epoch: 1 [9250/60000 (15%)]	Loss: 2.235105
Train Epoch: 1 [9300/60000 (16%)]	Loss: 2.983631
Train Epoch: 1 [9350/60000 (16%)]	Loss: 2.443490
Train Epoch: 1 [9400/60000 (16%)]	Loss: 2.536649
Train Epoch: 1 [9450/60000 (16%)]	Loss: 2.554959
Train Epoch: 1 [9500

Train Epoch: 1 [16750/60000 (28%)]	Loss: 1.945506
Train Epoch: 1 [16800/60000 (28%)]	Loss: 3.426509
Train Epoch: 1 [16850/60000 (28%)]	Loss: 2.623998
Train Epoch: 1 [16900/60000 (28%)]	Loss: 1.900130
Train Epoch: 1 [16950/60000 (28%)]	Loss: 2.560344
Train Epoch: 1 [17000/60000 (28%)]	Loss: 2.785554
Train Epoch: 1 [17050/60000 (28%)]	Loss: 1.502676
Train Epoch: 1 [17100/60000 (28%)]	Loss: 2.528768
Train Epoch: 1 [17150/60000 (29%)]	Loss: 1.863443
Train Epoch: 1 [17200/60000 (29%)]	Loss: 2.589135
Train Epoch: 1 [17250/60000 (29%)]	Loss: 3.618897
Train Epoch: 1 [17300/60000 (29%)]	Loss: 2.992162
Train Epoch: 1 [17350/60000 (29%)]	Loss: 1.723577
Train Epoch: 1 [17400/60000 (29%)]	Loss: 1.891919
Train Epoch: 1 [17450/60000 (29%)]	Loss: 2.462422
Train Epoch: 1 [17500/60000 (29%)]	Loss: 3.132294
Train Epoch: 1 [17550/60000 (29%)]	Loss: 2.676400
Train Epoch: 1 [17600/60000 (29%)]	Loss: 1.560700
Train Epoch: 1 [17650/60000 (29%)]	Loss: 2.417823
Train Epoch: 1 [17700/60000 (30%)]	Loss: 2.110831


Train Epoch: 1 [24950/60000 (42%)]	Loss: 1.025915
Train Epoch: 1 [25000/60000 (42%)]	Loss: 3.192042
Train Epoch: 1 [25050/60000 (42%)]	Loss: 1.594761
Train Epoch: 1 [25100/60000 (42%)]	Loss: 2.488214
Train Epoch: 1 [25150/60000 (42%)]	Loss: 1.745486
Train Epoch: 1 [25200/60000 (42%)]	Loss: 2.707242
Train Epoch: 1 [25250/60000 (42%)]	Loss: 2.295367
Train Epoch: 1 [25300/60000 (42%)]	Loss: 2.240561
Train Epoch: 1 [25350/60000 (42%)]	Loss: 2.361475
Train Epoch: 1 [25400/60000 (42%)]	Loss: 3.128645
Train Epoch: 1 [25450/60000 (42%)]	Loss: 1.637009
Train Epoch: 1 [25500/60000 (42%)]	Loss: 2.799720
Train Epoch: 1 [25550/60000 (43%)]	Loss: 2.683041
Train Epoch: 1 [25600/60000 (43%)]	Loss: 2.839473
Train Epoch: 1 [25650/60000 (43%)]	Loss: 2.174706
Train Epoch: 1 [25700/60000 (43%)]	Loss: 1.169433
Train Epoch: 1 [25750/60000 (43%)]	Loss: 2.983115
Train Epoch: 1 [25800/60000 (43%)]	Loss: 2.934541
Train Epoch: 1 [25850/60000 (43%)]	Loss: 2.576325
Train Epoch: 1 [25900/60000 (43%)]	Loss: 2.578900


Train Epoch: 1 [33150/60000 (55%)]	Loss: 3.615031
Train Epoch: 1 [33200/60000 (55%)]	Loss: 2.838298
Train Epoch: 1 [33250/60000 (55%)]	Loss: 3.124187
Train Epoch: 1 [33300/60000 (56%)]	Loss: 3.467735
Train Epoch: 1 [33350/60000 (56%)]	Loss: 4.048872
Train Epoch: 1 [33400/60000 (56%)]	Loss: 3.882712
Train Epoch: 1 [33450/60000 (56%)]	Loss: 2.440131
Train Epoch: 1 [33500/60000 (56%)]	Loss: 2.080773
Train Epoch: 1 [33550/60000 (56%)]	Loss: 2.754134
Train Epoch: 1 [33600/60000 (56%)]	Loss: 3.702142
Train Epoch: 1 [33650/60000 (56%)]	Loss: 1.982768
Train Epoch: 1 [33700/60000 (56%)]	Loss: 1.167317
Train Epoch: 1 [33750/60000 (56%)]	Loss: 3.942066
Train Epoch: 1 [33800/60000 (56%)]	Loss: 2.945643
Train Epoch: 1 [33850/60000 (56%)]	Loss: 2.370112
Train Epoch: 1 [33900/60000 (56%)]	Loss: 3.253590
Train Epoch: 1 [33950/60000 (57%)]	Loss: 2.533374
Train Epoch: 1 [34000/60000 (57%)]	Loss: 2.632288
Train Epoch: 1 [34050/60000 (57%)]	Loss: 2.795201
Train Epoch: 1 [34100/60000 (57%)]	Loss: 2.383190


Train Epoch: 1 [41350/60000 (69%)]	Loss: 4.147037
Train Epoch: 1 [41400/60000 (69%)]	Loss: 3.082244
Train Epoch: 1 [41450/60000 (69%)]	Loss: 1.371300
Train Epoch: 1 [41500/60000 (69%)]	Loss: 2.614472
Train Epoch: 1 [41550/60000 (69%)]	Loss: 3.886782
Train Epoch: 1 [41600/60000 (69%)]	Loss: 1.369542
Train Epoch: 1 [41650/60000 (69%)]	Loss: 3.256238
Train Epoch: 1 [41700/60000 (70%)]	Loss: 4.037656
Train Epoch: 1 [41750/60000 (70%)]	Loss: 2.304949
Train Epoch: 1 [41800/60000 (70%)]	Loss: 3.137400
Train Epoch: 1 [41850/60000 (70%)]	Loss: 3.373992
Train Epoch: 1 [41900/60000 (70%)]	Loss: 1.769817
Train Epoch: 1 [41950/60000 (70%)]	Loss: 3.215611
Train Epoch: 1 [42000/60000 (70%)]	Loss: 4.380465
Train Epoch: 1 [42050/60000 (70%)]	Loss: 4.722750
Train Epoch: 1 [42100/60000 (70%)]	Loss: 2.685276
Train Epoch: 1 [42150/60000 (70%)]	Loss: 2.288316
Train Epoch: 1 [42200/60000 (70%)]	Loss: 1.007621
Train Epoch: 1 [42250/60000 (70%)]	Loss: 2.703808
Train Epoch: 1 [42300/60000 (70%)]	Loss: 1.667388


Train Epoch: 1 [49550/60000 (83%)]	Loss: 2.216639
Train Epoch: 1 [49600/60000 (83%)]	Loss: 2.981045
Train Epoch: 1 [49650/60000 (83%)]	Loss: 2.657011
Train Epoch: 1 [49700/60000 (83%)]	Loss: 4.977817
Train Epoch: 1 [49750/60000 (83%)]	Loss: 3.629218
Train Epoch: 1 [49800/60000 (83%)]	Loss: 4.793130
Train Epoch: 1 [49850/60000 (83%)]	Loss: 2.925933
Train Epoch: 1 [49900/60000 (83%)]	Loss: 3.106675
Train Epoch: 1 [49950/60000 (83%)]	Loss: 5.104890
Train Epoch: 1 [50000/60000 (83%)]	Loss: 3.062166
Train Epoch: 1 [50050/60000 (83%)]	Loss: 3.499479
Train Epoch: 1 [50100/60000 (84%)]	Loss: 2.443806
Train Epoch: 1 [50150/60000 (84%)]	Loss: 2.446835
Train Epoch: 1 [50200/60000 (84%)]	Loss: 1.262188
Train Epoch: 1 [50250/60000 (84%)]	Loss: 2.558575
Train Epoch: 1 [50300/60000 (84%)]	Loss: 5.074514
Train Epoch: 1 [50350/60000 (84%)]	Loss: 4.099364
Train Epoch: 1 [50400/60000 (84%)]	Loss: 2.181319
Train Epoch: 1 [50450/60000 (84%)]	Loss: 4.502198
Train Epoch: 1 [50500/60000 (84%)]	Loss: 2.280692


Train Epoch: 1 [57750/60000 (96%)]	Loss: 3.082692
Train Epoch: 1 [57800/60000 (96%)]	Loss: 3.690936
Train Epoch: 1 [57850/60000 (96%)]	Loss: 2.969610
Train Epoch: 1 [57900/60000 (96%)]	Loss: 5.056574
Train Epoch: 1 [57950/60000 (97%)]	Loss: 5.559714
Train Epoch: 1 [58000/60000 (97%)]	Loss: 3.002325
Train Epoch: 1 [58050/60000 (97%)]	Loss: 2.142268
Train Epoch: 1 [58100/60000 (97%)]	Loss: 2.567738
Train Epoch: 1 [58150/60000 (97%)]	Loss: 2.750252
Train Epoch: 1 [58200/60000 (97%)]	Loss: 1.399550
Train Epoch: 1 [58250/60000 (97%)]	Loss: 3.518047
Train Epoch: 1 [58300/60000 (97%)]	Loss: 2.731811
Train Epoch: 1 [58350/60000 (97%)]	Loss: 2.653485
Train Epoch: 1 [58400/60000 (97%)]	Loss: 2.188731
Train Epoch: 1 [58450/60000 (97%)]	Loss: 4.024837
Train Epoch: 1 [58500/60000 (98%)]	Loss: 2.996370
Train Epoch: 1 [58550/60000 (98%)]	Loss: 2.909110
Train Epoch: 1 [58600/60000 (98%)]	Loss: 3.732906
Train Epoch: 1 [58650/60000 (98%)]	Loss: 3.111708
Train Epoch: 1 [58700/60000 (98%)]	Loss: 2.886787
